In [1]:
from pathlib import Path
import tomllib
from oauthlib.oauth2 import BackendApplicationClient
from requests_oauthlib import OAuth2Session
from blizzapi import Character, BlizzardAPIURLs, CharacterData

In [2]:
client_filepath = Path(r"D:\Programming\github-repos\nop-raid-bot\clientdetails.toml")
with open(client_filepath, "rb") as client_file:
    client_data = tomllib.load(client_file)

CLIENT_ID = client_data["client"]["id"]
CLIENT_SECRET = client_data["client"]["secret"]

TOKEN_URL = "https://oauth.battle.net/token"

In [3]:
client = BackendApplicationClient(client_id=CLIENT_ID)
oauth = OAuth2Session(client=client, state="blah123")
token = oauth.fetch_token(token_url=TOKEN_URL, client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
token

{'access_token': 'EU0dR3G5sspjE6Ohggj14w1msA0yD5cmUy',
 'token_type': 'bearer',
 'expires_in': 86399,
 'sub': '1492badc8bfe49469af56f9adf787596',
 'expires_at': 1730062647.6787403}

In [4]:
testchar = Character("Aptodk", "Draenor")
blizz_urls = BlizzardAPIURLs()
chardata = CharacterData(testchar, blizz_urls, oauth)
chardata.get_current_raid_data("The War Within", "Nerub-ar Palace", "Heroic")

'No data found for Nerub-ar Palace [Heroic]'

In [5]:
chardata.get_current_enchants(True)

"Enchants:\n- 9 / 9\n- Back: Enchanted: +355 Avoidance :quality3:\n- Chest: Enchanted: Crystalline Radiance :quality3:\n- Wrist: Enchanted: +650 Speed :quality3:\n- Legs: Enchanted: +930 Agility/Strength & +895 Stamina :quality3:\n- Feet: Enchanted: Plainsrunner's Breeze :quality2:\n- Ring 1: Enchanted: +190 Mastery :quality3:\n- Ring 2: Enchanted: +190 Versatility :quality3:\n- Main Hand: Enchanted: Rune of the Fallen Crusader\n- Off Hand: Enchanted: Rune of Razorice"

In [ ]:
equipment = oauth.get(blizz_urls.get_equipment(testchar)).json()
equipment

In [ ]:
[item["slot"]["name"] for item in equipment["equipped_items"]]

In [ ]:
equipment["equipped_items"][16]["enchantments"]

In [ ]:
raid_query = blizz_urls.get_raids(testchar)
raid_query

In [ ]:
response = oauth.get(raid_query)
response

In [ ]:
response.headers

In [ ]:
response.json()

In [ ]:
response.json()["expansions"]

In [ ]:
[data["expansion"]["name"] for data in response.json()["expansions"]]

In [73]:
expansions_list = [data["expansion"]["name"] for data in response.json()["expansions"]]
exp_idx = expansions_list.index("The War Within")
instance_names = [data["instance"]["name"] for data in response.json()["expansions"][exp_idx]["instances"]]
ins_idx = instance_names.index("Nerub-ar Palace")
difficulities = [data["difficulty"]["name"] for data in response.json()["expansions"][exp_idx]["instances"][ins_idx]["modes"]]
difficulty_index = difficulities.index("Normal")

In [ ]:
response.json()["expansions"][exp_idx]["instances"][ins_idx]["modes"][difficulty_index]["progress"]